### Mirar repeticions (amb mínim 1, 2, 3 o 4 columnes valuades iguals)

In [ ]:
import pandas as pd
import numpy as np

# 1. Carreguem les dades
df = pd.read_csv('train.csv') # 'test.csv'

# 2. Preprocessing
cols_to_exclude = [] # 'ID', 'Surename
if df.columns[0].startswith("Unnamed") or df.iloc[0,0] == 1:
    df_clean = df.iloc[:, 1:].drop(columns=[c for c in cols_to_exclude if c in df.columns])
else:
    df_clean = df.drop(columns=[c for c in cols_to_exclude if c in df.columns])

n_rows = len(df_clean)
print(f"Analitzant {n_rows} files i {len(df_clean.columns)} columnes...")

# --- MATRIUS ---
# mismatch_matrix (booleana): True si hi ha conflicte (A != B)
mismatch_matrix = np.zeros((n_rows, n_rows), dtype=bool)

# match_count_matrix (enter): Compta quantes coincidències exactes (A == B) hi ha
match_count_matrix = np.zeros((n_rows, n_rows), dtype=int)

for col in df_clean.columns:
    values = df_clean[col].values

    # Obtenim valors (v) i màscara de validesa (not_na)
    if pd.api.types.is_numeric_dtype(df_clean[col]):
        v = values
        not_na = ~np.isnan(v)
    else:
        codes, _ = pd.factorize(values, use_na_sentinel=True)
        v = codes
        not_na = (v != -1)

    # Broadcasting
    both_have_data = not_na[:, None] & not_na[None, :]
    values_equal = (v[:, None] == v[None, :])

    # 1. Acumulem Conflictes (OR lògic)
    # Si ja hi havia conflicte, es manté. Si n'hi ha un de nou, es marca.
    mismatch_matrix |= (both_have_data & (~values_equal))

    # 2. Sumem Coincidències (Suma aritmètica)
    # Convertim el booleà a enter (True=1, False=0) i sumem
    matches_in_this_col = (both_have_data & values_equal).astype(int)
    match_count_matrix += matches_in_this_col

# --- ANÀLISI DE RESULTATS ---

# Només ens interessa el triangle superior per no duplicar parelles
tri_upper = np.triu(np.ones((n_rows, n_rows), dtype=bool), k=1)

# Base: No hi ha conflictes
no_conflicts = (~mismatch_matrix) & tri_upper

print("\n--- RESULTATS DE L'ANÀLISI ---")
print(f"Parelles sense cap conflicte (però potser sense dades en comú): {np.sum(no_conflicts)}")

# Provem diferents llindars (Thresholds)
thresholds = [1, 2, 3, 4]

for t in thresholds:
    # Condició: No conflictes I (coincidències >= t)
    valid_pairs = no_conflicts & (match_count_matrix >= t)
    num_pairs = np.sum(valid_pairs)

    print(f"\n[LLINDAR {t}] Mínim {t} columnes idèntiques:")
    print(f"-> {num_pairs} parelles trobades.")

    if num_pairs > 0 and num_pairs < 10:
        # Si són poques, mostrem quines columnes coincideixen en el primer cas
        indices_i, indices_j = np.where(valid_pairs)
        idx1, idx2 = indices_i[0], indices_j[0]
        row1 = df_clean.iloc[idx1]
        row2 = df_clean.iloc[idx2]
        # Mirem quines columnes coincideixen
        common_cols = []
        for c in df_clean.columns:
            val1 = row1[c]
            val2 = row2[c]
            # Comprovació ràpida si són iguals i no nuls
            if pd.notna(val1) and pd.notna(val2) and val1 == val2:
                common_cols.append(c)
        print(f"   Exemple (Fila {idx1} vs {idx2}) coincideixen en: {common_cols}")

Analitzant 7000 files i 23 columnes...

--- RESULTATS DE L'ANÀLISI ---
Parelles sense cap conflicte (però potser sense dades en comú): 1332

[LLINDAR 1] Mínim 1 columnes idèntiques:
-> 1332 parelles trobades.

[LLINDAR 2] Mínim 2 columnes idèntiques:
-> 1327 parelles trobades.

[LLINDAR 3] Mínim 3 columnes idèntiques:
-> 1299 parelles trobades.

[LLINDAR 4] Mínim 4 columnes idèntiques:
-> 1227 parelles trobades.


### Comprovar per grups

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx

# 1. Carreguem les dades
df = pd.read_csv('train.csv')

# Configurem columnes a excloure
cols_to_exclude = []

# Neteja inicial per tenir el dataframe de treball
if df.columns[0].startswith("Unnamed") or df.iloc[0,0] == 1:
    df_clean = df.iloc[:, 1:].drop(columns=[c for c in cols_to_exclude if c in df.columns])
else:
    df_clean = df.drop(columns=[c for c in cols_to_exclude if c in df.columns])

n_rows = len(df_clean)
print(f"Analitzant {n_rows} files...")

# --- CONFIGURACIÓ ---
MIN_MATCHES = 3  # Mínim de columnes idèntiques

# --- PAS 1: Construcció de Matrius ---
mismatch_matrix = np.zeros((n_rows, n_rows), dtype=bool)
match_count_matrix = np.zeros((n_rows, n_rows), dtype=int)

print("Calculant coincidències...")
for col in df_clean.columns:
    values = df_clean[col].values

    if pd.api.types.is_numeric_dtype(df_clean[col]):
        v = values
        not_na = ~np.isnan(v)
    else:
        codes, _ = pd.factorize(values, use_na_sentinel=True)
        v = codes
        not_na = (v != -1)

    both_have_data = not_na[:, None] & not_na[None, :]
    values_equal = (v[:, None] == v[None, :])

    mismatch_matrix |= (both_have_data & (~values_equal))
    match_count_matrix += (both_have_data & values_equal).astype(int)

# --- PAS 2: Grafs ---
# Enllaç si: NO conflicte I MÍNIM 3 coincidències
adjacency = (~mismatch_matrix) & (match_count_matrix >= MIN_MATCHES)
np.fill_diagonal(adjacency, False)

G = nx.Graph()
rows_with_connections = np.where(adjacency)
for i, j in zip(rows_with_connections[0], rows_with_connections[1]):
    if i < j:
        G.add_edge(i, j)

connected_groups = list(nx.connected_components(G))

# --- PAS 3: Classificació i LLISTAT ---
clean_groups = []
ambiguous_groups = []

for group in connected_groups:
    indices = list(group)
    # Comprovem conflictes interns (clique)
    group_mismatch = mismatch_matrix[np.ix_(indices, indices)]
    np.fill_diagonal(group_mismatch, False)

    if np.any(group_mismatch):
        ambiguous_groups.append(sorted(indices))
    else:
        clean_groups.append(sorted(indices))

# --- RESULTATS: LLISTA D'ÍNDEXS ---
print("\n" + "="*60)
print(f"RESUM DE GRUPS TROBATS (Mínim {MIN_MATCHES} coincidències)")
print("="*60)

print(f"\n--- GRUPS NETS ({len(clean_groups)}) ---")
print("Aquests grups són segurs per fusionar (tots coincideixen entre ells).")
if len(clean_groups) == 0:
    print("   (Cap grup net trobat)")
else:
    for i, g in enumerate(clean_groups):
        # Imprimim llista formatada
        print(f"Grup {i+1}: {g}")

print(f"\n--- GRUPS AMBIGUS/CONFLICTIUS ({len(ambiguous_groups)}) ---")
print("Aquests grups tenen connexions però contradiccions internes (NO fusionar automàticament).")
if len(ambiguous_groups) == 0:
    print("   (Cap grup conflictiu trobat)")
else:
    for i, g in enumerate(ambiguous_groups):
        print(f"Grup {i+1}: {g}")

Analitzant 7000 files...
Calculant coincidències...

RESUM DE GRUPS TROBATS (Mínim 3 coincidències)

--- GRUPS NETS (32) ---
Aquests grups són segurs per fusionar (tots coincideixen entre ells).
Grup 1: [np.int64(146), np.int64(4416)]
Grup 2: [np.int64(309), np.int64(6731)]
Grup 3: [np.int64(358), np.int64(850)]
Grup 4: [np.int64(645), np.int64(2073)]
Grup 5: [np.int64(794), np.int64(1886)]
Grup 6: [np.int64(929), np.int64(4310)]
Grup 7: [np.int64(976), np.int64(3373)]
Grup 8: [np.int64(989), np.int64(1556)]
Grup 9: [np.int64(1129), np.int64(3038)]
Grup 10: [np.int64(1217), np.int64(5723)]
Grup 11: [np.int64(1233), np.int64(1618)]
Grup 12: [np.int64(1369), np.int64(5670)]
Grup 13: [np.int64(1537), np.int64(6764)]
Grup 14: [np.int64(1567), np.int64(5175)]
Grup 15: [np.int64(1980), np.int64(5494)]
Grup 16: [np.int64(2126), np.int64(2385)]
Grup 17: [np.int64(2268), np.int64(3081)]
Grup 18: [np.int64(2433), np.int64(4191)]
Grup 19: [np.int64(2482), np.int64(2633)]
Grup 20: [np.int64(2512),

### Codi bó

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx

# 1. Carreguem i preparem
df = pd.read_csv('train.csv')
cols_to_exclude = []

if df.columns[0].startswith("Unnamed") or df.iloc[0,0] == 1:
    df_clean = df.iloc[:, 1:].drop(columns=[c for c in cols_to_exclude if c in df.columns])
else:
    df_clean = df.drop(columns=[c for c in cols_to_exclude if c in df.columns])

n_rows = len(df_clean)
print(f"Analitzant {n_rows} files...")

# --- CONFIGURACIÓ ---
MIN_MATCHES = 3  # Llindar mínim per considerar connexió

# --- PAS 1: Matrius (Igual que abans) ---
mismatch_matrix = np.zeros((n_rows, n_rows), dtype=bool)
match_count_matrix = np.zeros((n_rows, n_rows), dtype=int)

print("Calculant coincidències...")
for col in df_clean.columns:
    values = df_clean[col].values
    if pd.api.types.is_numeric_dtype(df_clean[col]):
        v = values; not_na = ~np.isnan(v)
    else:
        codes, _ = pd.factorize(values, use_na_sentinel=True)
        v = codes; not_na = (v != -1)

    both_have_data = not_na[:, None] & not_na[None, :]
    values_equal = (v[:, None] == v[None, :])

    mismatch_matrix |= (both_have_data & (~values_equal))
    match_count_matrix += (both_have_data & values_equal).astype(int)

# --- PAS 2: Grafs i Grups ---
adjacency = (~mismatch_matrix) & (match_count_matrix >= MIN_MATCHES)
np.fill_diagonal(adjacency, False)

G = nx.Graph()
rows_with_connections = np.where(adjacency)
for i, j in zip(rows_with_connections[0], rows_with_connections[1]):
    if i < j: G.add_edge(i, j)

connected_groups = list(nx.connected_components(G))

# --- PAS 3: Separació Nets vs Conflictius ---
clean_groups = []
ambiguous_groups = []

for group in connected_groups:
    indices = list(group)
    group_mismatch = mismatch_matrix[np.ix_(indices, indices)]
    np.fill_diagonal(group_mismatch, False)

    if np.any(group_mismatch):
        ambiguous_groups.append(list(indices))
    else:
        clean_groups.append(list(indices))

# --- PAS 4 (NOU): Rescatar parelles dins dels grups conflictius ---
rescued_pairs = []

print(f"\nProcessant {len(ambiguous_groups)} grups conflictius per rescatar parelles...")

for group in ambiguous_groups:
    # 1. Busquem totes les connexions vàlides DINS del grup
    possible_edges = []
    for i in range(len(group)):
        for j in range(i + 1, len(group)):
            u, v = group[i], group[j]
            # Si estan connectats (compleixen criteri i no tenen conflicte directe)
            if adjacency[u, v]:
                weight = match_count_matrix[u, v]
                possible_edges.append((weight, u, v))

    # 2. Ordenem per "pes": Primer les parelles que tenen MÉS coincidències
    # Així prioritzem la unió més forta
    possible_edges.sort(key=lambda x: x[0], reverse=True)

    # 3. Selecció "voraz" (Greedy)
    # Si unim A amb B, ja no podem unir A amb C ni B amb C.
    taken_nodes = set()

    for w, u, v in possible_edges:
        if u not in taken_nodes and v not in taken_nodes:
            # Confirmem una última vegada que no tenen conflicte (per seguretat)
            if not mismatch_matrix[u, v]:
                rescued_pairs.append([u, v])
                taken_nodes.add(u)
                taken_nodes.add(v)

# --- RESULTATS FINALS ---
print("\n" + "="*60)
print("INFORME FINAL DE PARELLES PER FUSIONAR")
print("="*60)

print(f"\n1. GRUPS NETS (Ja eren perfectes): {len(clean_groups)} grups")
# Convertim grups nets a llista de llistes per uniformitat
final_merge_list = clean_groups + rescued_pairs

print(f"2. PARELLES RESCATADES de grups conflictius: {len(rescued_pairs)} parelles")
if len(rescued_pairs) > 0:
    print("   (Hem triat la millor parella dins del grup i descartat la resta)")
    for p in rescued_pairs[:5]: # Mostrem 5 exemples
        print(f"   -> Rescatada parella: {p} (Coincidències: {match_count_matrix[p[0], p[1]]})")

print("-" * 60)
print(f"TOTAL PARELLES/GRUPS A FUSIONAR: {len(final_merge_list)}")
print("-" * 60)

# Llistat complet per copiar i enganxar si vols
# print(final_merge_list)

Analitzant 7000 files...
Calculant coincidències...

Processant 21 grups conflictius per rescatar parelles...

INFORME FINAL DE PARELLES PER FUSIONAR

1. GRUPS NETS (Ja eren perfectes): 32 grups
2. PARELLES RESCATADES de grups conflictius: 236 parelles
   (Hem triat la millor parella dins del grup i descartat la resta)
   -> Rescatada parella: [np.int64(180), np.int64(2276)] (Coincidències: 14)
   -> Rescatada parella: [np.int64(2149), np.int64(4237)] (Coincidències: 12)
   -> Rescatada parella: [np.int64(2300), np.int64(4979)] (Coincidències: 11)
   -> Rescatada parella: [np.int64(4997), np.int64(5808)] (Coincidències: 11)
   -> Rescatada parella: [np.int64(3132), np.int64(1136)] (Coincidències: 11)
------------------------------------------------------------
TOTAL PARELLES/GRUPS A FUSIONAR: 268
------------------------------------------------------------


In [ ]:
import pandas as pd
import numpy as np
import networkx as nx

# --- CONFIGURACIÓ ---
MIN_MATCHES = 2  # Seguretat: Mínim 3 coincidències
MAX_LOOPS = 5    # Per seguretat, per evitar bucles infinits (normalment amb 2 o 3 n'hi ha prou)

# 1. Carreguem les dades
df_original = pd.read_csv('train.csv')
cols_to_exclude = []
if df_original.columns[0].startswith("Unnamed") or df_original.iloc[0,0] == 1:
    cols_ignore = cols_to_exclude + [df_original.columns[0]]
else:
    cols_ignore = cols_to_exclude

# Treballem amb una còpia
df_current = df_original.copy()

print(f"INICI: {len(df_current)} files.")

for loop in range(1, MAX_LOOPS + 1):
    print(f"\n--- RONDDA DE FUSIÓ {loop} ---")

    # Preparar dades per anàlisi (sense columnes ignorades)
    df_comp = df_current.drop(columns=[c for c in cols_ignore if c in df_current.columns])
    n_rows = len(df_comp)

    # 1. MATRIUS
    mismatch_matrix = np.zeros((n_rows, n_rows), dtype=bool)
    match_count_matrix = np.zeros((n_rows, n_rows), dtype=int)

    # Optimització: convertim a numpy arrays abans del bucle de columnes
    # Això accelera molt el procés
    print("   -> Analitzant coincidències...")
    for col in df_comp.columns:
        values = df_comp[col].values
        if pd.api.types.is_numeric_dtype(df_comp[col]):
            v = values; not_na = ~np.isnan(v)
        else:
            codes, _ = pd.factorize(values, use_na_sentinel=True)
            v = codes; not_na = (v != -1)

        both_have_data = not_na[:, None] & not_na[None, :]
        values_equal = (v[:, None] == v[None, :])

        mismatch_matrix |= (both_have_data & (~values_equal))
        match_count_matrix += (both_have_data & values_equal).astype(int)

    # 2. GRAF
    adjacency = (~mismatch_matrix) & (match_count_matrix >= MIN_MATCHES)
    np.fill_diagonal(adjacency, False)

    G = nx.Graph()
    rows_with_connections = np.where(adjacency)
    for i, j in zip(rows_with_connections[0], rows_with_connections[1]):
        if i < j: G.add_edge(i, j)

    connected_groups = list(nx.connected_components(G))

    if len(connected_groups) == 0:
        print("   -> Cap nova parella trobada. ATUREM EL PROCÉS.")
        break

    # 3. SELECCIÓ DE PARELLES (Nets + Rescatats)
    pairs_to_merge = []

    for group in connected_groups:
        indices = list(group)
        # Comprovem conflictes interns
        group_mismatch = mismatch_matrix[np.ix_(indices, indices)]
        np.fill_diagonal(group_mismatch, False)

        if not np.any(group_mismatch):
            # GRUP NET: Fusionem en cadena (0 amb 1, resultat amb 2...)
            # Afegim com a parelles seqüencials
            sorted_idx = sorted(indices)
            base_idx = sorted_idx[0]
            for other_idx in sorted_idx[1:]:
                pairs_to_merge.append((base_idx, other_idx))
        else:
            # GRUP CONFLICTIU: Rescatem parelles
            # Lògica "Greedy" dins del grup
            possible_edges = []
            for i in range(len(indices)):
                for j in range(i + 1, len(indices)):
                    u, v = indices[i], indices[j]
                    if adjacency[u, v]: # Si connecten
                         possible_edges.append((match_count_matrix[u, v], u, v))

            possible_edges.sort(key=lambda x: x[0], reverse=True)
            taken = set()
            for w, u, v in possible_edges:
                if u not in taken and v not in taken:
                    if not mismatch_matrix[u, v]:
                        pairs_to_merge.append((u, v))
                        taken.add(u); taken.add(v)

    print(f"   -> Identificades {len(pairs_to_merge)} fusions per fer.")

    if len(pairs_to_merge) == 0:
        print("   -> No s'han pogut rescatar parelles vàlides. FI.")
        break

    # 4. EXECUTAR FUSIONS I ACTUALITZAR DATAFRAME
    # Important: Com que els índexs canvien si esborrem, primer fusionem i marquem, després esborrem

    rows_to_drop = set()
    # Ordenem per índex per prioritzar mantenir els de dalt
    pairs_to_merge.sort(key=lambda x: x[0])

    count_merged = 0
    for keep_idx, drop_idx in pairs_to_merge:
        # A la ronda actual, fem servir els índexs positius (iloc)
        # Però hem d'anar amb compte: si hem modificat el DF, els iloc canvien?
        # NO, si no fem drop fins al final del bucle.

        if keep_idx in rows_to_drop or drop_idx in rows_to_drop:
            continue

        # FUSIÓ: Combine first
        # row_keep es queda amb les seves dades, i omple NA amb row_drop
        row_keep = df_current.iloc[keep_idx]
        row_drop = df_current.iloc[drop_idx]

        merged_series = row_keep.combine_first(row_drop)

        # Actualitzem el DataFrame en memòria (al mateix lloc que keep_idx)
        df_current.iloc[keep_idx] = merged_series

        rows_to_drop.add(drop_idx)
        count_merged += 1

    print(f"   -> Fusionades {count_merged} parelles. Eliminant redundants...")

    # Esborrem les files i RESTEGEM l'índex per la següent volta
    df_current = df_current.drop(df_current.index[list(rows_to_drop)]).reset_index(drop=True)
    print(f"   -> Files restants: {len(df_current)}")

# RESULTAT FINAL
print("\n" + "="*50)
print(f"PROCÉS COMPLETAT.")
print(f"Files inicials: {len(df_original)}")
print(f"Files finals: {len(df_current)}")
print(f"Total eliminades: {len(df_original) - len(df_current)}")
print("="*50)

# Guardar
df_current.to_csv('train_max_dedup.csv', index=False)

INICI: 3000 files.

--- RONDDA DE FUSIÓ 1 ---
   -> Analitzant coincidències...
   -> Cap nova parella trobada. ATUREM EL PROCÉS.

PROCÉS COMPLETAT.
Files inicials: 3000
Files finals: 3000
Total eliminades: 0


### Afegim eliminar

In [ ]:
import pandas as pd
import numpy as np

# 1. Carreguem les dades
df = pd.read_csv('train.csv')

# Guardem una còpia de treball
df_clean = df.copy()

# 2. Definim columnes a ignorar per la COMPARACIÓ (però les conservarem al resultat)
cols_ignore = ['ID', 'Surname']
# Si la primera columna és un índex sense nom, l'afegim a ignorar
if df.columns[0].startswith("Unnamed") or df.iloc[0,0] == 1:
    cols_ignore.append(df.columns[0])

# Columnes vàlides per comparar
cols_to_compare = [c for c in df.columns if c not in cols_ignore]

print(f"Iniciant procés de neteja sobre {len(df)} files...")

# --- PAS 1: Càlcul matricial ràpid per trobar candidats (com abans) ---
# Això ens estalvia fer un bucle lent Python de tots contra tots

# Preparem dades només de comparació
df_comp = df_clean[cols_to_compare]
n_rows = len(df_comp)

# Matrius booleanes
mismatch_matrix = np.zeros((n_rows, n_rows), dtype=bool)
match_found_matrix = np.zeros((n_rows, n_rows), dtype=bool)

for col in df_comp.columns:
    values = df_comp[col].values

    if pd.api.types.is_numeric_dtype(df_comp[col]):
        v = values
        not_na = ~np.isnan(v)
    else:
        codes, _ = pd.factorize(values, use_na_sentinel=True)
        v = codes
        not_na = (v != -1)

    both_have_data = not_na[:, None] & not_na[None, :]
    values_equal = (v[:, None] == v[None, :])

    mismatch_matrix |= (both_have_data & (~values_equal))
    match_found_matrix |= (both_have_data & values_equal)

# Identifiquem parelles candidates (Triangle superior per no duplicar)
tri_upper = np.triu(np.ones((n_rows, n_rows), dtype=bool), k=1)
is_candidate = (~mismatch_matrix) & match_found_matrix & tri_upper
indices_i, indices_j = np.where(is_candidate)

# --- PAS 2: Fusió i Eliminació (Consolidació) ---
# Iterem sobre els candidats trobats per fusionar informació

rows_to_drop = set()
count_merged = 0

# Ordenem per índex per assegurar ordre (processar primer les de dalt)
# Convertim a llista de tuples per iterar
candidates = sorted(zip(indices_i, indices_j), key=lambda x: x[0])

for idx_keep, idx_drop in candidates:
    # Si la fila que volem mantenir (idx_keep) ja ha estat marcada per esborrar
    # (perquè era la "repetida" d'una anterior), passem.
    if idx_keep in rows_to_drop:
        continue

    # Si la fila candidata a esborrar ja està marcada, passem
    if idx_drop in rows_to_drop:
        continue

    # --- FUSIÓ ---
    # Omplim els NAs de la fila original amb els valors de la fila repetida
    # Utilitzem combine_first: prioritza df_clean.iloc[idx_keep], omple forats amb idx_drop
    original_row = df_clean.iloc[idx_keep]
    duplicate_row = df_clean.iloc[idx_drop]

    # Fem el merge (això retorna una nova sèrie)
    merged_row = original_row.combine_first(duplicate_row)

    # Actualitzem la fila "mestra" al DataFrame
    df_clean.iloc[idx_keep] = merged_row

    # Marquem la segona per eliminar
    rows_to_drop.add(idx_drop)
    count_merged += 1

# --- PAS 3: Esborrar les files redundants ---
df_final = df_clean.drop(index=list(rows_to_drop))

print(f"\nResultats:")
print(f"- Parelles detectades inicialment: {len(indices_i)}")
print(f"- Files fusionades i eliminades: {count_merged}")
print(f"- Files originals: {len(df)}")
print(f"- Files finals: {len(df_final)}")

# Guardem l'arxiu net
df_final.to_csv('train_cleaned.csv', index=False)
print("\nArxiu guardat com 'train_cleaned.csv'")